In [1]:
from keras.models import Sequential  
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras import layers, models, optimizers, losses
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os 
import glob

In [2]:
# CARREGAMENTO DOS DATASETS

''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''
img_dir = "C:/Users/marci/Desktop/Projeto mestrado/Images Dataset/Batch 06.04 Images"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

'''   
    Carregando e tratando o arquivo CSV
'''
eggClass = pd.read_csv("RealData.csv")
cList = eggClass.to_numpy()

print(len(data))
print(cList.shape)

5100
(5100, 1)


In [ ]:
subset = []
values = []
for i in range(len(cList)):
    if cList[i] == 0:
        subset.append(i)
    if cList[i] == 1:
        values.append(i)

randomIndex = np.random.choice(values, 1373, replace=False)

for j in range(len(randomIndex)):
    subset.append(randomIndex[j])

print(subset)
print(len(subset))


In [ ]:
ct1 = 0
ct2 = 0
for z in range(len(subset)):
    if cList[subset[z]] == 0:
        ct1 += 1
    if cList[subset[z]] == 1:
        ct2 += 1
    
print(ct1)
print(ct2)

In [ ]:
# Dataset Completo
# Comentar seção caso não seja necessário

#ind = np.arange(len(data))
train, test = train_test_split(subset, test_size=0.2, random_state=42)

#
dataTrain = []
dataTest = []
classTrain = []
classTest = []

#
for i in range(len(train)):

    dataTrain.append(data[train[i]])
    classTrain.append(cList[train[i]])

for j in range(len(test)):

    dataTest.append(data[test[j]])
    classTest.append(cList[test[j]])

#
dataTrain = np.array(dataTrain)
dataTest = np.array(dataTest)
classTrain = np.array(classTrain)
classTest = np.array(classTest)

print('img train', len(dataTrain))
print('img test', len(dataTest))
print('class train', len(classTrain))
print('class test', len(classTest))


In [6]:
del data
del cList
del subset
del values
del randomIndex

In [7]:
'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

dataTrain = dataTrain.astype('float32')
dataTest = dataTest.astype('float32')

'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

dataTrain /= 255
dataTest /= 255

In [3]:
# SHALLOW NETWORK

classifier = Sequential()

classifier.add(Conv2D(32, (3,3), input_shape = (512, 512, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [3]:
# ALEXNET

model = Sequential()

model.add(Conv2D(96, (11, 11), strides=4, activation='relu', input_shape = (512, 512, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# TREINO ALEXNET

board = TensorBoard(log_dir='./logs30AlexSubset50x50')
model.fit(dataTrain, classTrain, batch_size = 32, epochs = 30, validation_data = (dataTest, classTest), callbacks=[board])

In [ ]:
# TREINO SHALLOWNET

board2 = TensorBoard(log_dir='./logsShallow')
classifier.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board2])

In [10]:
#model.save_weights('AlexNet30Subset50x50.weights.h5')
#model2.save_weights('mobileNet10ep.weights.h5')
#classifier.save_weights('shallownet.weights.h5')

In [4]:
# Carregando pesos

model.load_weights('AlexNet30Subset50x50.weights.h5')
#classifier.load_weights('shallownet.weights.h5')

In [5]:
npData = np.array(data)
alexPredict = model.predict(npData)
#shallowPredict = classifier.predict(npData)


160/160 [==============================] - 168s 1s/step


In [8]:
npAlex = np.array(alexPredict)
#npShallow = np.array(shallowPredict)

print(npAlex)
print('-----------------')
#print(npShallow)

[[0.805369  ]
 [0.8288535 ]
 [0.83847976]
 ...
 [0.813279  ]
 [0.8383563 ]
 [0.8837195 ]]
-----------------


In [24]:
groundTruth = np.ravel(cList)
maxValue = 0
thresh = 0

for i in np.arange(0, 1.00, 0.01):
    AlexClass = (npAlex >= i).astype(int)
    AlexClass = np.ravel(AlexClass)
    hits = 0

    for j in range(len(groundTruth)):
        if AlexClass[j] == groundTruth[j]:
            hits += 1

    if hits >= maxValue:
        maxValue = hits
        thresh = i
    
print(maxValue)
print(thresh)   

        


3728
0.73


In [19]:
AlexClass = (npAlex >= 0.5).astype(int)
#ShallowClass = (npShallow >= 0.5).astype(int)
AlexClass = np.ravel(AlexClass)
#ShallowClass = np.ravel(ShallowClass)
print(AlexClass)
#print(ShallowClass)

[1 1 1 ... 1 1 1]
